In [1]:
# Задание 1
# Загрузите данные и определите разницу в конверсиях между контрольной и тестовой группами. 
# Сгруппируйте данные по колонке treatment и вычислите среднее значение и количество для столбца conversion.


import pandas as pd
# загрузите данные
data = pd.read_csv("yandex_plus.csv")

# агрегация данных
conv = (data
    .groupby("treatment")['conversion']
    .agg(["mean","count"])
)
print(conv)

              mean  count
treatment                
control     0.4056   5000
treatment1  0.4932   5000


In [2]:
#Задание 3 
# Обучите алгоритм uplift-деревьев, предварительно разделив выборку на тренировочную и валидационную в соотношении 70 на 30. 
# Определите модель UpliftRandomForestClassifier, передав в аргументы параметры модели. 
# Напишите алгоритм подбора оптимальных параметров модели на основе метрики Uplift@10%, которую необходимо максимизировать.
# 
# В этом задании вам нужно:
#   разделить данные на тренировочную (70%) и валидационную (30%) выборки;
#   создать и обучить модель UpliftRandomForestClassifier с различными параметрами;
#   оценить качество модели с помощью метрики Uplift@10% (эффект воздействия для топ-10% клиентов);
#   найти оптимальные параметры модели, которые максимизируют метрику Uplift@10%.
# 
# В модели UpliftRandomForestClassifier можно регулировать следующие параметры:
# evaluationFunction
# функция оценки качества разбиения (KL, ED, Chi и др.);
# max_depth
# максимальная глубина деревьев;
# n_estimators
# количество деревьев в ансамбле.



import pandas as pd
from sklearn.model_selection import train_test_split
from causalml.inference.tree import UpliftRandomForestClassifier
from itertools import product

# загружаем данные
data = pd.read_csv("yandex_plus.csv")

# разделение на признаки и целевую переменную
X = data.drop(['conversion'], axis=1)
y = data['conversion']
 

# разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,
                                                random_state=42)

max_uplift = 0
top_percent = 0.1
best_params = {}

eval_functions = ['KL', 'ED', 'Chi']
max_depths = [6, 7, 8]
n_estimators = [50, 100]

results = []

for eval_f, m_depth, n_est in product(eval_functions, max_depths, n_estimators):
    uplift_model = UpliftRandomForestClassifier(
        control_name='control',
        evaluationFunction=eval_f,
        max_depth=m_depth,
        n_estimators=n_est,
        random_state=42
    )

    uplift_model.fit(
        X_train.drop(columns=['treatment']).values,
        treatment=X_train['treatment'].values,
        y=y_train.values
    )

    uplift_preds = uplift_model.predict(X_val.drop(columns=['treatment']), full_output=False)
    uplift_preds_df = (pd.DataFrame(uplift_preds, columns = ['uplift_score'])
                ['uplift_score']
                .sort_values(ascending=False)
    )

    mean_uplift = (uplift_preds_df
                [:int(len(uplift_preds_df) * top_percent)]
                .mean()
    )

    results.append({
        'eval_function': eval_f,
        'max_depth': m_depth,
        'n_estimators': n_est,
        'mean_uplift@10%': mean_uplift
    })

    if mean_uplift > max_uplift:
        max_uplift = mean_uplift
        best_params = {
            "eval_function": eval_f,
            "max_depth": m_depth,
            "n_estimators": n_est
        }

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print("Best params:", best_params)
print("Best uplift@10%:", max_uplift)


Failed to import duecredit due to No module named 'duecredit'


Best params: {'eval_function': 'ED', 'max_depth': 8, 'n_estimators': 100}
Best uplift@10%: 0.29201481331494156


In [ ]:
# Задание 5
# Переобучите модель с оптимальными параметрами, полученными в прошлом задании, и сделайте прогноз на валидационной выборке.

# инициализация модели
uplift_model = UpliftRandomForestClassifier(
        control_name='control',
        evaluationFunction='ED',
        max_depth=8,
        n_estimators=100,
        random_state=42
    )

# обучение модели
uplift_model.fit(
        X_train.drop(columns=['treatment']).values,
        treatment=X_train['treatment'].values,
        y=y_train.values
    )

# получение прогнозов
uplift_preds = uplift_model.predict(X_val.drop(columns=['treatment']), full_output=False)


In [5]:
uplift_preds

array([[-0.08078938],
       [ 0.28884885],
       [ 0.25927697],
       ...,
       [ 0.18553644],
       [ 0.14426173],
       [ 0.04160616]], shape=(3000, 1))

In [6]:
# Задание 6
# Выберите пользователей со значением uplift больше 0.2. 
# Для удобства прогнозы были переведены в формат pandas.DataFrame с единственной колонкой uplift_score. 
# Сохраните полученные значения в CSV-формате с названием файла top_users.csv.

uplift_preds_df = pd.DataFrame(uplift_preds, columns=['uplift_score'])


top_users = uplift_preds_df.query("uplift_score > 0.2")

top_users.to_csv("top_users.csv")

In [7]:
top_users

,uplift_score
1,0.288849
2,0.259277
3,0.202616
9,0.279016
10,0.236600
...,...
2986,0.276581
2988,0.299405
2989,0.208250
2995,0.206984
